In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user
from tests.test_full_week import walk_through_week


# Test: Full Week

In [ ]:
# mock_user = get_user(os.environ["JAREDS_ATHLETE_ID"])
# response = walk_through_week(mock_user, "2024-09-01")

# for training_week in response:
#     print(training_week.total_mileage)

# Test: One Off

In [ ]:
# from src.daily_pipeline import new_training_week_pipeline, mid_week_update_pipeline
# from tests.test_full_week import gen_helper

# mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
# response = gen_helper(
#     mock_user=mock_user,
#     date_str="2024-09-09",
#     func=mid_week_update_pipeline,
# )
# pprint(response.sessions)

# Evals: New Training Week

In [ ]:
# path = 'tests/artifacts/new_training_week_errors'
# files = os.listdir(path)

# for file in sorted(files):
#     print(f"Loading {file}")
#     with open(os.path.join(path, file), "r") as f:
#         results = json.load(f)
#     print(sum(result['error'] for result in results) / len(results))

In [ ]:
# for result in results:
#     print(result['user']['email'])
#     print("target:", result['recommended_weekly_mileage'])
#     print("actual:", result['generated_weekly_mileage'])
#     print("error:", result['error'])
#     print("." * 100)

# Analytics

In [ ]:
# from src.supabase_client import list_users, get_training_week

# for user in list_users():
#     training_week = get_training_week(user.athlete_id)
#     print(user.email)
#     for session in training_week.sessions:
#         print(session)
#     print(training_week.total_mileage)
#     print("-" * 100)

# Manual New Training Week

In [4]:
# from src.supabase_client import list_users

# list_users()

In [ ]:
# from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
# from src.auth_manager import get_strava_client
# from src.supabase_client import get_user
# from src.constants import COACH_ROLE
# from src.new_training_week import generate_new_training_week
# import os
# from pprint import pprint

# user = get_user()
# strava_client = get_strava_client(user.athlete_id)
# sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {user.preferences}\n"
# activities_df = get_activities_df(strava_client)
# day_of_week_summaries = get_day_of_week_summaries(activities_df)
# weekly_summaries = get_weekly_summaries(activities_df)
# response = generate_new_training_week(
#     sysmsg_base=sysmsg_base,
#     weekly_summaries=weekly_summaries,
#     day_of_week_summaries=day_of_week_summaries,
# )

# pprint(response.sessions)

In [ ]:
# from src.supabase_client import upsert_training_week

# upsert_training_week(user.athlete_id, response)

# Trigger Lambda E2E

In [2]:
# import requests
# import os

# url = "https://lwg77yq7dd.execute-api.us-east-1.amazonaws.com/prod/signup"

# response = requests.post(url, json={
#     "trigger_test_key": os.environ["TRIGGER_TEST_KEY"]
# })

# response.json()

In [6]:
from src.auth_manager import get_strava_client
from src.email_manager import send_email
from src.supabase_client import list_users

html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>TrackFlow Update</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 600px;
            margin: 40px auto;
            background-color: #fff;
            border-radius: 10px;
            box-shadow: 0 0 15px rgba(0, 0, 0, 0.1);
            overflow: hidden;
        }
        .title {
            text-align: center;
            font-size: 36px;
            color: #333;
            padding: 20px;
        }
        .content {
            padding: 20px;
            color: #555;
            line-height: 1.6;
        }
        .content p {
            margin-bottom: 20px;
        }
        .highlight {
            color: #5A86D5;
            font-weight: bold;
        }
        .footer {
            text-align: center;
            padding: 20px;
            background-color: #5A86D5;
            color: white;
            font-size: 14px;
        }
        .footer a {
            color: #fff;
            text-decoration: none;
        }
        .footer a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1 class="title">TrackFlow 🏃‍♂️🎯</h1>
        <div class="content">
            <p>Hey friends,</p>
            <p>As the fall race season wraps up, I want to thank you all for being early testers of TrackFlow and for letting me spam your inbox with notifications.</p>
            <p><span class="highlight">TrackFlow will be going offline — temporarily.</span> You'll no longer receive training plan updates via email, but if you'd like to keep using it through the winter, just let me know.</p>
            <p>Good news: I'm planning to re-launch in a few months with a new format (hint: no more email!).</p>
            <p>Thanks for all the support,<br>Jamie</p>
        </div>
        <div class="footer">
            <p>TrackFlow 🏃‍♂️🎯 | Powered by the Strava API & OpenAI</p>
        </div>
    </div>
</body>
</html>
"""

for user in list_users():
    strava_client = get_strava_client(user.athlete_id)
    athlete = strava_client.get_athlete()
    send_email(
        subject="TrackFlow Offline for the Winter 😴",
        html_content=html,
        to={"email": user.email, "name": f"{athlete.firstname} {athlete.lastname}"},
    )

No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
No rates present in response headers
